<a href="https://colab.research.google.com/github/QaziSaim/Fine-Tune-Projects/blob/main/Attention_Mechenisum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Luckly I already had saved my tokenizer now i am going to load it

In [1]:
import numpy as np
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

In [2]:
df = pd.read_csv('/content/eng_to_fra.csv')

In [3]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(df.eng_texts)
eng_sequence = eng_tokenizer.texts_to_sequences(df.eng_texts)

In [5]:
fra_tokenizer = Tokenizer()
fra_tokenizer.fit_on_texts(df.fra_texts)
fra_sequence = fra_tokenizer.texts_to_sequences(df.fra_texts)

In [7]:
eng_vocab_size = len(eng_tokenizer.word_index) +1
fra_vocab_size = len(fra_tokenizer.word_index) + 1


In [8]:
max_eng_len = max([len(x) for x in eng_sequence])
max_fra_len = max([len(x) for x in fra_sequence])

In [11]:
max_eng_len,max_fra_len

(44, 57)